In [ ]:
%%bash
python3.13 -m venv /Users/jethro/Documents/Code/GitOak/MultiFractal/multifractal_env
source /Users/jethro/Documents/Code/GitOak/MultiFractal/multifractal_env/bin/activate

In [ ]:
%%bash
cd /Users/jethro/Documents/Code/GitOak/MultiFractal/

Step 2 will be to install dependencies

In [ ]:
%%bash
pip install numpy pandas matplotlib mfdaf nibabel

After that, we need to extract time series data

In [ ]:
import os
import nibabel as nib
import numpy as np

# Define data directory
data_dir = "/Users/jethro/Documents/Code/GitOak/MultiFractal"
output_dir = "/Users/jethro/Documents/Code/GitOak/MultiFractal/extracted_time_series"
os.makedirs(output_dir, exist_ok=True)

# Function to extract mean time-series for each subject
def extract_time_series(subject_id, mask_path=None):
    # Adjust filename pattern to match actual file names
    subject_path = os.path.join(data_dir, subject_id, "func", 
                                f"{subject_id}_task-bilateralfingertapping_echo-1_bold.nii.gz")
    
    if not os.path.exists(subject_path):
        raise FileNotFoundError(f"File not found: {subject_path}")
    
    img = nib.load(subject_path)
    data = img.get_fdata()
    
    # If mask is provided, apply it
    if mask_path:
        mask = nib.load(mask_path).get_fdata()
        data = data[mask > 0]
    
    # Average time-series across voxels
    time_series = data.mean(axis=(0, 1, 2))  # Adjust if using a mask
    return time_series

# Loop through subjects
for sub in range(2, 14):  # sub-02 to sub-13
    sub_id = f"sub-{sub:02d}"
    print(f"Processing {sub_id}...")
    try:
        time_series = extract_time_series(sub_id)
        np.savetxt(os.path.join(output_dir, f"{sub_id}_time_series.csv"), time_series, delimiter=",")
    except FileNotFoundError as e:
        print(e)

Following this, we proceed with MultiFractal Analysis on the data using analytics software

In [ ]:
# install dependenies for MultiFractal Analysis - MF-DFA
! pip install MFDFA

Then we use MFDFA to analyze the extracted data. Results for Hurst exponents (hurst), scaling exponents (tau), singularity strengths (alpha), and multifractal spectra (f(α)) are saved as .csv files. Plots of the multifractal spectrum are saved for each subject.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from MFDFA import MFDFA
from scipy.interpolate import UnivariateSpline
from scipy.stats import linregress

# Define directories
time_series_dir = "/Users/jethro/Documents/Code/GitOak/MultiFractal/extracted_time_series"
output_dir = "/Users/jethro/Documents/Code/GitOak/MultiFractal/multifractal_results"
os.makedirs(output_dir, exist_ok=True)

# MF-DFA parameters
scale_min, scale_max = 10, 500  # Minimum and maximum scales

def compute_Hq(Fq, scales):
    num_q = Fq.shape[1]
    Hq = np.zeros(num_q)
    log_scales = np.log2(scales)
    
    for i in range(num_q):
        log_Fq = np.log2(Fq[:, i])
        # Handle any invalid values
        valid = np.isfinite(log_Fq)
        if np.sum(valid) < 2:
            Hq[i] = np.nan
            continue
        # Perform linear regression
        slope, _, _, _, _ = linregress(log_scales[valid], log_Fq[valid])
        Hq[i] = slope
    return Hq

def compute_singularity_spectrum(q, tau):
    # Remove NaN values
    valid = ~np.isnan(tau)
    q = q[valid]
    tau = tau[valid]
    
    # Fit a cubic spline to tau(q)
    spline = UnivariateSpline(q, tau, k=3, s=0)
    # Compute derivative of tau with respect to q
    alpha = spline.derivative()(q)
    # Compute f(α) as q*α - τ(q)
    f_alpha = q * alpha - tau
    return alpha, f_alpha

def perform_mfdfa(file_path, output_path):
    # Load time-series data
    time_series = np.loadtxt(file_path)
    
    # Define scales and q_values
    scales = np.logspace(np.log10(scale_min), np.log10(scale_max), num=20, base=10).astype(int)
    q_values = np.arange(-5, 6, 1)
    q_values = q_values[q_values != 0]  # Exclude q=0
    
    # Perform MF-DFA
    # Unpack the returned fluctuation functions
    Fq = MFDFA(time_series, scales, q=q_values)[0]
    
    # Verify Fq is a NumPy array
    print(f"Fq shape: {Fq.shape}")
    
    # Compute Hq
    Hq = compute_Hq(Fq, scales)
    
    # Calculate tau(q)
    tau = q_values * Hq - 1
    
    # Compute singularity spectrum
    alpha, f_alpha = compute_singularity_spectrum(q_values, tau)
    
    # Save results
    np.savetxt(os.path.join(output_path, "Hq.csv"), Hq, delimiter=",")
    np.savetxt(os.path.join(output_path, "tau.csv"), tau, delimiter=",")
    np.savetxt(os.path.join(output_path, "alpha.csv"), alpha, delimiter=",")
    np.savetxt(os.path.join(output_path, "f_alpha.csv"), f_alpha, delimiter=",")
    
    # Plot the multifractal spectrum
    plt.figure(figsize=(8, 6))
    plt.plot(alpha, f_alpha, '-o')
    plt.xlabel('Singularity Strength (α)')
    plt.ylabel('Multifractal Spectrum (f(α))')
    plt.title(f'Multifractal Spectrum for {os.path.basename(file_path)}')
    plt.grid()
    plt.savefig(os.path.join(output_path, "multifractal_spectrum.png"))
    plt.close()

# Loop through extracted time-series files
for file_name in os.listdir(time_series_dir):
    if file_name.endswith(".csv"):
        subject_id = file_name.split("_")[0]

The output should produce fluctuation and correlation functions and diagrams for the processed data. The fact that the code seems to have run quickly without any output is concerning, though. The output folder remains empty at the moment, but I'm hopeful that the files are simply rendering and should populate shortly.

It's looking like the process is simply taking a long time to run, and because VSCode handles python scripts through a kernel solver, they don't show up in the processes page of actvity monitor, but the network acivity still shows a long-running python task which appears to still be working.